# Exploration of the Dataset
This notebook performs the **first stage of data wrangling** for the NYC 311 Service Requests dataset.  
The goal is to **collect, clean, and prepare representative samples** from the raw data to support further **exploration and analysis** in later steps.

We connect to the official [NYC Open Data API](https://data.cityofnewyork.us/), download data in manageable chunks per quarter,  
and generate proportional samples across boroughs — ensuring that the dataset remains statistically representative while keeping it computationally efficient.

This notebook lays the groundwork for:
- **Merging and unifying** large raw datasets  
- **Sampling** data per borough and time period  

In [1]:
# Imports
import pandas as pd
import logging
import re
from libs.fetcher import fetch_count_of_grouping, fetch_all_samples_from_plan
from libs.utils import generate_quarters, month_ranges
from libs.calculator import calc_sample_size
from itertools import chain

## ⚙️ Constants and Configuration

The following section defines key constants used throughout this notebook:
- **BASE_URL:** the NYC Open Data API endpoint for 311 Service Requests  
- **DEFAULT_SINCE / DEFAULT_UNTIL:** default year range for sampling  
- **TARGET_SAMPLE:** target number of records for each quarterly sample  
- **DEFAULT_DB_PATH / DEFAULT_TABLE:** optional configuration for local DuckDB storage  
- **MAX_RETRIES / TIMEOUT / BASE_DELAY:** network parameters for reliable API requests  
- **SELECT_COLUMNS:** the list of columns (fields) to retrieve from the API  
- **data_sets:** any additional CSV datasets used for contextual enrichment (e.g., housing, demographic, or rent data)


In [2]:
# Constants
BASE_URL = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv"
DEFAULT_SINCE = 2024
DEFAULT_UNTIL = 2025
TARGET_SAMPLE = 10_000
DEFAULT_DB_PATH = "./mydb.duckdb"
DEFAULT_TABLE = "nyc311_2024_2025"
MAX_RETRIES = 5
TIMEOUT = 60  # seconds
BASE_DELAY = 2.0  # seconds

SELECT_COLUMNS = [
    "unique_key", "created_date", "closed_date", "agency", "agency_name", 
    "complaint_type", "descriptor", "location_type", "incident_zip", 
    "incident_address", "street_name", "cross_street_1", "cross_street_2",
    "intersection_street_1", "intersection_street_2", "address_type", "city", 
    "landmark", "facility_type", "status", "due_date", "resolution_description", 
    "resolution_action_updated_date", "community_board", "bbl", "borough", 
    "x_coordinate_state_plane", "y_coordinate_state_plane", "open_data_channel_type",
    "park_facility_name", "park_borough", "vehicle_type", "taxi_company_borough", 
    "taxi_pick_up_location", "bridge_highway_name", "bridge_highway_direction", 
    "road_ramp", "bridge_highway_segment", "latitude", "longitude", "location"
]

data_sets = [
     "data/medianAskingRent_All.csv",
]

##  1. Workflow Overview

1. **Generate month range:**  
   For each year between the selected start and end years, we create `(start, end)` date pairs.  
   First we create quarterly ranges and then split them into the months.
   Example: `2024-01-01T00:00:00` → `2024-01-31T23:59:59` (M1 2024)

2. **Fetch Borough Counts:**  
   Using the Socrata API, we retrieve the total number of service requests per `borough` within each quarter.  
   → Output: a list or DataFrame with columns  
   `['borough', 'total']`

3. **Compute Sampling Plan:**  
   Based on each borough’s proportion of total records, we calculate how many samples to take per borough:  
   $$
   n_i = N_\text{sample} \times \frac{\text{total}_i}{\text{total}_\text{overall}}
   $$
   The result is a sampling plan with one `sample_size` value per borough.

4. **Fetch Random Samples:**  
   For each borough, we randomly pull `sample_size` records from the corresponding month using the openly accessible API.  
   - Data is retrieved via the `.csv` endpoint (faster than JSON).  
   - A random `$offset` and a random choice, acsending or descending is used in sampling ensure randomness.

5. **Combine All Quarters:**  
   The sampled data from all boroughs and quarters are concatenated into a single combined DataFrame using  
   `pd.concat(all_quarters, ignore_index=True)`.

---

## ⚙️ 2. Key Functions

| Function | Description |
|-----------|--------------|
| `generate_quarters(start_year, end_year)` | Generates quarterly date ranges |
| `fetch_count_of_grouping(BASE_URL, group_by, start, end)` | Retrieves counts per group (e.g., borough) |
| `calc_sample_size(count_result)` | Computes proportional sample sizes |
| `fetch_random_sample(...)` | Fetches a random subset for one borough and quarter |
| `fetch_all_samples_from_plan(...)` | Iterates over boroughs and collects their samples |
| `fetch_all_quarters(...)` *(optional)* | Runs the entire pipeline across all quarters |

---

In [3]:
# Fetch sample of datasets and parse to Data Frame 

# 1. generate the time ranges:
quarters = generate_quarters(DEFAULT_SINCE, DEFAULT_UNTIL)

data_frames = []
months = []

for start, end in quarters:
    months.append(month_ranges(start, end))
    logging.info(f"Quarter from {start} to {end}")

df_plan = None

print(months)

for start, end in chain.from_iterable(months):
    count_result = fetch_count_of_grouping(BASE_URL, "borough", start, end)
    df_plan = calc_sample_size(count_result, TARGET_SAMPLE)

    df_311_calls = fetch_all_samples_from_plan(
                    BASE_URL=BASE_URL,
                    selectors=SELECT_COLUMNS,
                    df_plan=df_plan,
                    group_by="borough", 
                    time_start=start,
                    time_end=end,
                    sleep_seconds=BASE_DELAY
    )
    df_311_calls["month_start"] = start
    df_311_calls["month_end"] = end
    data_frames.append(df_311_calls)

df_all_calls = pd.concat(data_frames, ignore_index=True)
logging.info(f"Total records fetched: {len(df_all_calls)}")
df_all_calls.to_csv("data/nyc_311_2024_2025_sample.csv", index=False)

[[('2024-01-01T00:00:00', '2024-02-01T00:00:00'), ('2024-02-01T00:00:00', '2024-03-01T00:00:00'), ('2024-03-01T00:00:00', '2024-03-31T23:59:59')], [('2024-04-01T00:00:00', '2024-05-01T00:00:00'), ('2024-05-01T00:00:00', '2024-06-01T00:00:00'), ('2024-06-01T00:00:00', '2024-06-30T23:59:59')], [('2024-07-01T00:00:00', '2024-08-01T00:00:00'), ('2024-08-01T00:00:00', '2024-09-01T00:00:00'), ('2024-09-01T00:00:00', '2024-09-30T23:59:59')], [('2024-10-01T00:00:00', '2024-11-01T00:00:00'), ('2024-11-01T00:00:00', '2024-12-01T00:00:00'), ('2024-12-01T00:00:00', '2024-12-31T23:59:59')], [('2025-01-01T00:00:00', '2025-02-01T00:00:00'), ('2025-02-01T00:00:00', '2025-03-01T00:00:00'), ('2025-03-01T00:00:00', '2025-03-31T23:59:59')], [('2025-04-01T00:00:00', '2025-05-01T00:00:00'), ('2025-05-01T00:00:00', '2025-06-01T00:00:00'), ('2025-06-01T00:00:00', '2025-06-30T23:59:59')], [('2025-07-01T00:00:00', '2025-08-01T00:00:00'), ('2025-08-01T00:00:00', '2025-09-01T00:00:00'), ('2025-09-01T00:00:00', '2

/Users/roberto/Git/DAW_NYC_data/libs/fetcher.py:98: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(all_samples, ignore_index=True)


<class 'pandas.core.series.Series'>
📥 Loading 2057 rows for borough = 'BRONX' ...
2057 rows loaded.
📥 Loading 3091 rows for borough = 'BROOKLYN' ...
3091 rows loaded.
📥 Loading 2086 rows for borough = 'MANHATTAN' ...
2086 rows loaded.
📥 Loading 2418 rows for borough = 'QUEENS' ...
2418 rows loaded.
📥 Loading 339 rows for borough = 'STATEN ISLAND' ...
339 rows loaded.
📥 Loading 9 rows for borough = 'Unspecified' ...
0 rows loaded.


/Users/roberto/Git/DAW_NYC_data/libs/fetcher.py:98: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(all_samples, ignore_index=True)


<class 'pandas.core.series.Series'>
📥 Loading 1952 rows for borough = 'BRONX' ...
1952 rows loaded.
📥 Loading 3063 rows for borough = 'BROOKLYN' ...
3063 rows loaded.
📥 Loading 2110 rows for borough = 'MANHATTAN' ...
2110 rows loaded.
📥 Loading 2499 rows for borough = 'QUEENS' ...
2499 rows loaded.
📥 Loading 368 rows for borough = 'STATEN ISLAND' ...
368 rows loaded.
📥 Loading 8 rows for borough = 'Unspecified' ...
0 rows loaded.


...

/Users/roberto/Git/DAW_NYC_data/libs/fetcher.py:98: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(all_samples, ignore_index=True)


<class 'pandas.core.series.Series'>
📥 Loading 2367 rows for borough = 'BRONX' ...
2367 rows loaded.
📥 Loading 2888 rows for borough = 'BROOKLYN' ...
2888 rows loaded.
📥 Loading 2069 rows for borough = 'MANHATTAN' ...
2069 rows loaded.
📥 Loading 2327 rows for borough = 'QUEENS' ...
2327 rows loaded.
📥 Loading 342 rows for borough = 'STATEN ISLAND' ...
342 rows loaded.
📥 Loading 7 rows for borough = 'Unspecified' ...
0 rows loaded.


/Users/roberto/Git/DAW_NYC_data/libs/fetcher.py:98: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(all_samples, ignore_index=True)


<class 'pandas.core.series.Series'>
📥 Loading 1731 rows for borough = 'BRONX' ...
0 rows loaded.
📥 Loading 2390 rows for borough = 'MANHATTAN' ...
0 rows loaded.
📥 Loading 2445 rows for borough = 'QUEENS' ...
0 rows loaded.
📥 Loading 3063 rows for borough = 'BROOKLYN' ...
0 rows loaded.
📥 Loading 8 rows for borough = 'Unspecified' ...
0 rows loaded.
📥 Loading 363 rows for borough = 'STATEN ISLAND' ...
0 rows loaded.
<class 'pandas.core.series.Series'>


/var/folders/sc/hk55c1v54g3dff0094vmwkb80000gn/T/ipykernel_4928/716405911.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_calls = pd.concat(data_frames, ignore_index=True)
